In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math

In [2]:
questURL = 'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=1\
           &max_rnk=1\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'
Quest = requests.get(questURL).json()
Quest

{'StudyFieldsResponse': {'APIVrs': '1.01.02',
  'DataVrs': '2020:07:23 00:12:14.048',
  'Expression': 'covid 19',
  'NStudiesAvail': 346614,
  'NStudiesFound': 2857,
  'MinRank': 1,
  'MaxRank': 1,
  'NStudiesReturned': 1,
  'FieldList': ['NCTId',
   'BriefTitle',
   'StatusVerifiedDate',
   'CompletionDate',
   'OrgFullName',
   'OrgClass',
   'Keyword',
   'LocationCity',
   'LocationCountry'],
  'StudyFields': [{'Rank': 1,
    'NCTId': ['NCT04395482'],
    'BriefTitle': ['Lung CT Scan Analysis of SARS-CoV2 Induced Lung Injury'],
    'StatusVerifiedDate': ['May 2020'],
    'CompletionDate': ['October 15, 2020'],
    'OrgFullName': ['University of Milano Bicocca'],
    'OrgClass': ['OTHER'],
    'Keyword': ['Lung injury', 'sars-covid-2', 'coronavirus infection'],
    'LocationCity': ['Bergamo',
     'Bergamo',
     'Ferrara',
     'Lecco',
     'Melzo',
     'Monza',
     'Rimini',
     'San Marino'],
    'LocationCountry': ['Italy',
     'Italy',
     'Italy',
     'Italy',
     'Ita

In [3]:
limit1 = Quest['StudyFieldsResponse']['NStudiesFound'] #make sure we don't ask for results outside of the scope of our query
limit1

2857

In [4]:
# Query the api in packets of 1000 then merge into a single dataframe

tempURL1 = 'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=1\
           &max_rnk=1000\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'   
tempL1 = requests.get(tempURL1).json()

tempURL2 = 'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=1001\
           &max_rnk=2000\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'
tempL2 = requests.get(tempURL2).json()                                                             

tempURL3 = f'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=2001\
           &max_rnk={limit1}\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'
tempL3 = requests.get(tempURL3).json()

Q1_tempL1 = tempL1['StudyFieldsResponse']['StudyFields']
Q1_tempL2 = tempL2['StudyFieldsResponse']['StudyFields']
Q1_tempL3 = tempL3['StudyFieldsResponse']['StudyFields']

In [5]:
Q1_tempDF1 = pd.DataFrame(Q1_tempL1)
Q1_tempDF2 = pd.DataFrame(Q1_tempL2)
Q1_tempDF3 = pd.DataFrame(Q1_tempL3)

In [6]:
#returns single list of col values from all 3 temp DF's 

def unListItems(col):
    lst_1 = [' ']*len(Q1_tempDF1[col])  # create empty list the length of 'Q1_tempDF1[col]'
    i = 0
    while i < len(Q1_tempDF1[col]):
        if len(Q1_tempDF1[col][i])==0:
            lst_1[i] = None
        else:
            lst_1[i] = Q1_tempDF1[col][i][0]
        i += 1
    lst_2 = [' ']*len(Q1_tempDF2[col])
    i = 0
    while i < len(Q1_tempDF2[col]):
        if len(Q1_tempDF2[col][i])==0:
            lst_2[i] = None
        else:
            lst_2[i] = Q1_tempDF2[col][i][0]
        i += 1      
    lst_3 = [' ']*len(Q1_tempDF3[col])
    i = 0
    while i < len(Q1_tempDF3[col]):
        if len(Q1_tempDF3[col][i])==0:
            lst_3[i] = None
        else:
            lst_3[i] = Q1_tempDF3[col][i][0]
        i += 1
    LIST = lst_1 + lst_2 + lst_3
    return LIST

In [7]:
#retrive unListed Items, populate Query1 DataFrame with query results from id, title, and date keys

ID = unListItems('NCTId')
Title = unListItems('BriefTitle')
VerifiedDate = unListItems('StatusVerifiedDate')
CompletionDate = unListItems('CompletionDate')
OrgFullName = unListItems('OrgFullName')
OrgClass = unListItems('OrgClass')
QueryType = ['covid 19']*len(ID)
Country = unListItems('LocationCountry')
City = unListItems('LocationCity')


DF_Q1 = pd.DataFrame()
DF_Q1.insert(0, 'ID', ID)
DF_Q1.insert(1, 'Title', Title)
DF_Q1.insert(2, 'Date Verified', VerifiedDate)
DF_Q1.insert(3, 'Completion Date', CompletionDate)
DF_Q1.insert(4, 'Organization', OrgFullName)
DF_Q1.insert(5, 'Organization Class', OrgClass)
DF_Q1.insert(6, 'Country', Country)
DF_Q1.insert(7, 'City', City)

In [8]:
groupID = DF_Q1.groupby('ID')

df = pd.DataFrame(groupID.count())

df = df.loc[df['Title']>1]

dupeList = []
for i in df.index:
    dupeList.append(i)
print(dupeList)

['NCT04244591', 'NCT04273581', 'NCT04285801', 'NCT04287686', 'NCT04291053', 'NCT04306497', 'NCT04317092', 'NCT04321421', 'NCT04321993', 'NCT04329650', 'NCT04329923', 'NCT04331795', 'NCT04332094', 'NCT04338347', 'NCT04339660', 'NCT04342221', 'NCT04346017', 'NCT04346043', 'NCT04346615', 'NCT04347369', 'NCT04347993', 'NCT04348214', 'NCT04348656', 'NCT04348942', 'NCT04350476', 'NCT04350593', 'NCT04351620', 'NCT04351763', 'NCT04352465', 'NCT04353674', 'NCT04354805', 'NCT04354870', 'NCT04355936', 'NCT04358510', 'NCT04358939', 'NCT04359290', 'NCT04359511', 'NCT04360278', 'NCT04361214', 'NCT04361643', 'NCT04362085', 'NCT04362189', 'NCT04363034', 'NCT04363866', 'NCT04365738', 'NCT04366115', 'NCT04369807', 'NCT04370236', 'NCT04371822', 'NCT04372628', 'NCT04375046', 'NCT04377620', 'NCT04379440', 'NCT04382066', 'NCT04383678', 'NCT04383886', 'NCT04385849', 'NCT04387214', 'NCT04389645', 'NCT04389671', 'NCT04391712', 'NCT04393038', 'NCT04393311', 'NCT04393324', 'NCT04394416', 'NCT04400812', 'NCT04401

In [9]:
query = 'coronavirus'
quest2URL = f'http://ClinicalTrials.gov/api/query/study_fields?expr={query}\
           &min_rnk=1\
           &max_rnk=1\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'
Quest2 = requests.get(quest2URL).json()
Quest2

{'StudyFieldsResponse': {'APIVrs': '1.01.02',
  'DataVrs': '2020:07:23 00:12:14.048',
  'Expression': 'coronavirus',
  'NStudiesAvail': 346614,
  'NStudiesFound': 1452,
  'MinRank': 1,
  'MaxRank': 1,
  'NStudiesReturned': 1,
  'FieldList': ['NCTId',
   'BriefTitle',
   'StatusVerifiedDate',
   'CompletionDate',
   'OrgFullName',
   'OrgClass',
   'Keyword',
   'LocationCity',
   'LocationCountry'],
  'StudyFields': [{'Rank': 1,
    'NCTId': ['NCT04279795'],
    'BriefTitle': ['Detection of 2019 Novel Coronavirus in Multiple Organ System and Its Relationship With Clinical Manifestations'],
    'StatusVerifiedDate': ['February 2020'],
    'CompletionDate': ['February 28, 2021'],
    'OrgFullName': ['Third Affiliated Hospital, Sun Yat-Sen University'],
    'OrgClass': ['OTHER'],
    'Keyword': ['2019 Novel Coronavirus'],
    'LocationCity': ['Guangzhou'],
    'LocationCountry': ['China']}]}}

In [10]:
limit2 = Quest2['StudyFieldsResponse']['NStudiesFound'] #make sure we don't ask for results outside of the scope of our query
limit2 

1452

In [11]:
# Query the api in packets of 1000 then merge into a single dataframe

tempURL1 = 'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=1\
           &max_rnk=1000\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'   
tempL1 = requests.get(tempURL1).json()

tempURL2 = f'http://ClinicalTrials.gov/api/query/study_fields?expr=covid+19\
           &min_rnk=1001\
           &max_rnk={limit2}\
           &fields=\
               NCTId,\
               BriefTitle,\
               StatusVerifiedDate,\
               CompletionDate,\
               OrgFullName,\
               OrgClass,\
               keyword,\
               LocationCity,\
               LocationCountry\
           &fmt=JSON'
tempL2 = requests.get(tempURL2).json()                                                             

Q2_tempL1 = tempL1['StudyFieldsResponse']['StudyFields']
Q2_tempL2 = tempL2['StudyFieldsResponse']['StudyFields']

In [12]:
Q2_tempDF1 = pd.DataFrame(Q2_tempL1)
Q2_tempDF2 = pd.DataFrame(Q2_tempL2)

In [13]:
#returns single list of col values from all 3 temp DF's 

def unListItemsQ2(col):
    lst_1 = [' ']*len(Q2_tempDF1[col])  # create empty list the length of 'Q1_tempDF1[col]'
    i = 0
    while i < len(Q2_tempDF1[col]):
        if len(Q2_tempDF1[col][i])==0:
            lst_1[i] = None
        else:
            lst_1[i] = Q2_tempDF1[col][i][0]
        i += 1
    lst_2 = [' ']*len(Q2_tempDF2[col])
    i = 0
    while i < len(Q2_tempDF2[col]):
        if len(Q2_tempDF2[col][i])==0:
            lst_2[i] = None
        else:
            lst_2[i] = Q2_tempDF2[col][i][0]
        i += 1      
    LIST = lst_1 + lst_2
    return LIST

In [14]:
#retrive unListed Items, populate Query1 DataFrame with query results from id, title, and date keys

ID = unListItemsQ2('NCTId')
Title = unListItemsQ2('BriefTitle')
VerifiedDate = unListItemsQ2('StatusVerifiedDate')
CompletionDate = unListItemsQ2('CompletionDate')
OrgFullName = unListItemsQ2('OrgFullName')
OrgClass = unListItemsQ2('OrgClass')
QueryType = ['coronavirus']*len(ID)
Country = unListItemsQ2('LocationCountry')
City = unListItemsQ2('LocationCity')

DF_Q2 = pd.DataFrame()
DF_Q2.insert(0, 'ID', ID)
DF_Q2.insert(1, 'Title', Title)
DF_Q2.insert(2, 'Date Verified', VerifiedDate)
DF_Q2.insert(3, 'Completion Date', CompletionDate)
DF_Q2.insert(4, 'Organization', OrgFullName)
DF_Q2.insert(5, 'Organization Class', OrgClass)
DF_Q2.insert(6, 'Country', Country)
DF_Q2.insert(7, 'City', City)

In [15]:
DF_QQ = pd.concat([DF_Q1,DF_Q2], ignore_index=True)
DF_QQ = DF_QQ.drop_duplicates()
DF_QQ.set_index('ID', inplace=True)
DF_QQ

,Title,Date Verified,Completion Date,Organization,Organization Class,Country,City
ID,,,,,,,
NCT04395482,Lung CT Scan Analysis of SARS-CoV2 Induced Lun...,May 2020,"October 15, 2020",University of Milano Bicocca,OTHER,Italy,Bergamo
NCT04395924,Maternal-foetal Transmission of SARS-Cov-2,June 2020,May 2021,Centre Hospitalier Régional d'Orléans,OTHER,France,Orléans
NCT04476940,COVID-19 Breastfeeding Guideline for African-A...,July 2020,June 2022,Meharry Medical College,OTHER,United States,Nashville
NCT04367805,COVID-19 Infection in Patients With Hepatocell...,April 2020,October 2020,"Centre Hospitalier Universitaire, Amiens",OTHER,France,Amiens
NCT04412265,Frailty in Elderly Patients With COVID-19,June 2020,"March 1, 2021",University of Milano Bicocca,OTHER,Italy,Monza
...,...,...,...,...,...,...,...
NCT04362969,COVID-EC Study: Clinical Characteristics of Pa...,May 2020,May 2020,"Hospital San Carlos, Madrid",OTHER,Ecuador,Guayaquil
NCT04431869,In-Utero Vascular Accidents in Neonates From C...,June 2020,July 2022,"University of Colorado, Denver",OTHER,United States,Aurora
NCT04402905,Observational Study in Diagnosed Patients COVI...,May 2020,"August 15, 2020",Centre de Recherches et d'Etude sur la Patholo...,OTHER,France,Paris


In [16]:
#Create a dataframe with keywords, BriefTitle and trial ID

Keywords1 = Q1_tempDF1[['NCTId', 'BriefTitle', 'Keyword']]
Keywords2 = Q1_tempDF2[['NCTId', 'BriefTitle', 'Keyword']]
Keywords3 = Q1_tempDF3[['NCTId', 'BriefTitle', 'Keyword']]
Keywords4 = Q2_tempDF1[['NCTId', 'BriefTitle', 'Keyword']]
Keywords5 = Q2_tempDF1[['NCTId', 'BriefTitle', 'Keyword']]

Keywords = pd.concat([Keywords1,Keywords2,Keywords3,Keywords4,Keywords5], ignore_index=True) 
Keywords.set_index('NCTId')


,BriefTitle,Keyword
NCTId,,
[NCT04395482],[Lung CT Scan Analysis of SARS-CoV2 Induced Lu...,"[Lung injury, sars-covid-2, coronavirus infect..."
[NCT04395924],[Maternal-foetal Transmission of SARS-Cov-2],"[Pregnancy, RT-PCR-COVID-19, SARS-CoV 2 serolo..."
[NCT04476940],[COVID-19 Breastfeeding Guideline for African-...,"[COVID-19, Exclusive Breastfeeding, Breastfeed..."
[NCT04367805],[COVID-19 Infection in Patients With Hepatocel...,"[Hepatocellular Carcinoma, COVID-19, Coronavir..."
[NCT04412265],[Frailty in Elderly Patients With COVID-19],"[Coronavirus, sars-covid-2, coronavirus infect..."
...,...,...
[NCT04348240],[Transmissibility and Viral Load of SARS-CoV-2...,"[Saliva, Coronavirus, SARS-CoV-2, Symptoms, CO..."
[NCT04400448],[Acceptance of Telemedicine During the COVID-1...,"[telemedicine, patient acceptance, covid-19]"
[NCT04445402],[Pediatrics HOT COVID-19 Database in NY Tristate],"[Immune System Disorder in Children, Immune Sy..."


In [17]:
id_list = []
for l in Keywords['NCTId']:
    l = l[0]
    id_list.append(l)
Keywords.insert(3, 'ID', id_list)

In [18]:
Keywords.drop('NCTId', axis=1, inplace=True)

In [19]:
Keywords.set_index('ID', inplace=True)

In [20]:
Keywords

,BriefTitle,Keyword
ID,,
NCT04395482,[Lung CT Scan Analysis of SARS-CoV2 Induced Lu...,"[Lung injury, sars-covid-2, coronavirus infect..."
NCT04395924,[Maternal-foetal Transmission of SARS-Cov-2],"[Pregnancy, RT-PCR-COVID-19, SARS-CoV 2 serolo..."
NCT04476940,[COVID-19 Breastfeeding Guideline for African-...,"[COVID-19, Exclusive Breastfeeding, Breastfeed..."
NCT04367805,[COVID-19 Infection in Patients With Hepatocel...,"[Hepatocellular Carcinoma, COVID-19, Coronavir..."
NCT04412265,[Frailty in Elderly Patients With COVID-19],"[Coronavirus, sars-covid-2, coronavirus infect..."
...,...,...
NCT04348240,[Transmissibility and Viral Load of SARS-CoV-2...,"[Saliva, Coronavirus, SARS-CoV-2, Symptoms, CO..."
NCT04400448,[Acceptance of Telemedicine During the COVID-1...,"[telemedicine, patient acceptance, covid-19]"
NCT04445402,[Pediatrics HOT COVID-19 Database in NY Tristate],"[Immune System Disorder in Children, Immune Sy..."


In [21]:
dups = Keywords.pivot_table(index=['ID'], aggfunc='size')
dups_list = []
for i, d in enumerate(dups):
    if d > 1:
        dups_list.append(dups.index[i])

In [23]:
Keywords = Keywords.reset_index()

In [24]:
dupLL = []
for d in dups_list:
    dupf = Keywords.loc[Keywords['ID']==d]
    l = len(dupf.index)
    dupL = ['']*l
    for i, x in enumerate(dupL):
        dupL[i] = dupf.index[i]
    dupLL.append(dupL)

for x in dupLL:
    Keywords.drop(x[1:], axis=0, inplace=True)
        

In [27]:
#Clean Up
Keywords = Keywords.reset_index()

In [30]:
#Remove empty Keyword fields
for i, k in enumerate(Keywords['Keyword']):
    if len(k) == 0:
        Keywords.drop(i, axis=0, inplace=True)

In [32]:
#Clean Up
Keywords = Keywords.reset_index()

In [33]:
Keywords

,level_0,index,ID,BriefTitle,Keyword
0,0,0,NCT04395482,[Lung CT Scan Analysis of SARS-CoV2 Induced Lu...,"[Lung injury, sars-covid-2, coronavirus infect..."
1,1,1,NCT04395924,[Maternal-foetal Transmission of SARS-Cov-2],"[Pregnancy, RT-PCR-COVID-19, SARS-CoV 2 serolo..."
2,2,2,NCT04476940,[COVID-19 Breastfeeding Guideline for African-...,"[COVID-19, Exclusive Breastfeeding, Breastfeed..."
3,3,3,NCT04367805,[COVID-19 Infection in Patients With Hepatocel...,"[Hepatocellular Carcinoma, COVID-19, Coronavir..."
4,4,4,NCT04412265,[Frailty in Elderly Patients With COVID-19],"[Coronavirus, sars-covid-2, coronavirus infect..."
...,...,...,...,...,...
1640,2844,3842,NCT04376944,[Barrier Measures to Limit Covid-19.Nosocomial...,[barrier measures]
1641,2845,3843,NCT04395768,[International ALLIANCE Study of Therapies to ...,"[COVID19, hydroxychloroquine, azithromycin, zi..."
1642,2846,3846,NCT04477993,[Ruxolitinib for Acute Respiratory Disorder Sy...,"[SARS-CoV2, Severe Acute Respiratory Syndrome ..."
1643,2850,3852,NCT04348240,[Transmissibility and Viral Load of SARS-CoV-2...,"[Saliva, Coronavirus, SARS-CoV-2, Symptoms, CO..."


In [45]:
all_words = pd.DataFrame()
wordsList = []
for k in Keywords['Keyword']:
    for word in k:
        wordsList.append(word)
all_words.insert(0, 'Word', wordsList)

wordsCount = all_words.pivot_table(index=['Word'], aggfunc='size')
wordsCount = wordsCount.sort_values(ascending=False)
wordCountDF = pd.DataFrame(wordsCount)
wordCountDF.rename(columns={0:'Count'}, inplace=True)
wordCountDF.head(50)

,Count
Word,
COVID-19,657
SARS-CoV-2,254
Coronavirus,138
Covid-19,92
ARDS,64
COVID,59
coronavirus,50
Hydroxychloroquine,47
COVID19,39
